In [ ]:
'''
face/eyes detection
抓脸
'''

import numpy as np
import cv2 as cv
face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml')
cap = cv.VideoCapture(0)
num = 0

while(True):
    # 一帧一帧捕捉
    ret, frame = cap.read()
    # 我们对帧的操作在这里
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        eyes_found = False
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray,1.1,2)
        for (ex,ey,ew,eh) in eyes:
            if not eyes_found: 
                eyes_found = True
                face_img = frame[y-10:y+h+10, x-10:x+w+10]
                image_name = './james/%d.jpg' % (num)
                cv.imwrite(image_name,face_img)
                num += 1
                cv.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
            cv.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
    cv.imshow('img',frame)
    if cv.waitKey(2) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

In [ ]:
'''
训练
'''
import os
import numpy as np
import cv2

#脸部检测函数
def face_detect_demo(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    face_detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    faces = face_detector.detectMultiScale(gray, 1.2, 6)
    # 如果未检测到面部，则返回原始图像
    if (len(faces) == 0):
        return None, None
    # 目前假设只有一张脸，xy为左上角坐标，wh为矩形的宽高
    (x, y, w, h) = faces[0]
    # 返回图像的脸部部分
    return gray[y:y + w, x:x + h], faces[0]

def ReFileName(dirPath):
    """
    :param dirPath: 文件夹路径
    :return:
    """
    # 对目录下的文件进行遍历
    faces=[]
    for file in os.listdir(dirPath):
        # 判断是否是文件
        if os.path.isfile(os.path.join(dirPath, file)) == True:
           c= os.path.basename(file)
           name = dirPath + '\\' + c
           img = cv2.imread(name)
           # 检测脸部
           face, rect = face_detect_demo(img)
           # 我们忽略未检测到的脸部
           if face is not None:
               # 将脸添加到脸部列表并添加相应的标签
               faces.append(face)
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    return faces

#David
dirDavid = r"C:\james_home\local_src\codebase\pyexec\opencv\face_recognition\david"#文件路径
Davidfaces=ReFileName(dirDavid)#调用函数
labelDavidfaces=np.array([0 for i in range(len(Davidfaces))])#标签处理

#Ning
dirNing = r"C:\james_home\local_src\codebase\pyexec\opencv\face_recognition\ning"#文件路径
Ningfaces=ReFileName(dirNing)#调用函数
labelNingfaces=np.array([1 for i in range(len(Ningfaces))])#标签处理

#Jamesfaces
dirMyfaces = r"C:\james_home\local_src\codebase\pyexec\opencv\face_recognition\james"#文件路径
Jamesfaces=ReFileName(dirMyfaces)#调用函数
labelJamesfaces=np.array([2 for i in range(len(Jamesfaces))])#标签处理

#Jingfaces
dirJing = r"C:\james_home\local_src\codebase\pyexec\opencv\face_recognition\jingjing"#文件路径
Jingfaces=ReFileName(dirJing)#调用函数
labelJingfaces=np.array([3 for i in range(len(Jingfaces))])#标签处理

#Rosefaces
dirRose = r"C:\james_home\local_src\codebase\pyexec\opencv\face_recognition\rose"#文件路径
Rosefaces=ReFileName(dirRose)#调用函数
labelRosefaces=np.array([4 for i in range(len(Rosefaces))])#标签处理

#拼接并打乱数据特征和标签
x=np.concatenate((Jamesfaces,Davidfaces,Ningfaces,Jingfaces, Rosefaces),axis=0)
y=np.concatenate((labelJamesfaces,labelDavidfaces,labelNingfaces,labelJingfaces, labelRosefaces),axis=0)

index = [i for i in range(len(y))] # test_data为测试数据
np.random.seed(1)
np.random.shuffle(index) # 打乱索引
train_data = x[index]
train_label = y[index]

#分类器
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.train(train_data, train_label)
# 保存训练数据
recognizer.write('train.yml')

In [ ]:
'''
预测
'''
import cv2
#人脸检测函数
def face_detect_demo(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    face_detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    eye_cascade = cv2.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml')
    faces = face_detector.detectMultiScale(gray, 1.2, 6)
    # 如果未检测到面部，则返回原始图像
    if (len(faces) == 0):
        return None, None
    # 目前假设只有一张脸，xy为左上角坐标，wh为矩形的宽高
    (x, y, w, h) = faces[0]
    roi_gray = gray[y:y+h, x:x+w]
    eyes = eye_cascade.detectMultiScale(roi_gray,1.1,2)
    if (len(eyes) == 0):
        return None, None 
    # 返回图像的脸部部分
    return gray[y:y + w, x:x + h], faces[0]

# 根据给定的人脸（x，y）坐标和宽度高度在图像上绘制矩形
def draw_rectangle(img, rect):
    (x, y, w, h) = rect#矩形框
    cv2.rectangle(img, (x, y), (x + w, y + h), (255, 255, 0), 2)

# 根据给定的人脸（x，y）坐标写出人名
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_COMPLEX, 1, (128, 128, 0), 2)

# 此函数识别图像中的人物并在脸部周围绘制一个矩形及其人名
facelabel = ["David", "Ning", "MaYun", "MaJingJing", "Rose"]#人物名

#导入训练结果
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('train.yml')#读取前文训练的结果

def predict(image):
 
    # 检测人脸区域
    face, rect = face_detect_demo(image)#face_detect_demo前面的人脸检测函数
    #print(rect)=[x,y,w,h]
    # 预测人脸名字
    if not face is None:
        label = recognizer.predict(face)
        print(label)#label[0]为名字，label[1]可信度数值越低，可信度越高（
        if label[1]<=70:
            # 获取由人脸识别器返回的相应标签的人名
            label_text = facelabel[label[0]]

            # 在检测到的脸部周围画一个矩形
            draw_rectangle(image, rect)
            # 标出预测的人名
            draw_text(image, label_text, rect[0], rect[1])
            # 返回预测的图像
            return image
        else:
            # 在检测到的脸部周围画一个矩形
            draw_rectangle(image, rect)
            # 标出预测的人名
            draw_text(image, "not find", rect[0], rect[1])
            # 返回预测的图像
            return image
    else:
        return image

cap = cv2.VideoCapture(0)
while(True):
    # 一帧一帧捕捉
    ret, frame = cap.read()
    # 我们对帧的操作在这里
    pred_img = predict(frame)
    cv2.imshow('img',pred_img)
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
'''
dlib test
面部标记识别
https://blog.csdn.net/qq_19707521/article/details/80366524
'''
from collections import OrderedDict
import numpy as np
import cv2
import dlib
import imutils
 
# 定义一个映射面部索引的字典
# 特定人脸区域的地标
FACIAL_LANDMARKS_IDXS = OrderedDict([
	("mouth", (48, 68)),
	("right_eyebrow", (17, 22)),
	("left_eyebrow", (22, 27)),
	("right_eye", (36, 42)),
	("left_eye", (42, 48)),
	("nose", (27, 36)),
	("jaw", (0, 17))
])
 
def rect_to_bb(rect):
        # 由DLIB预测一个边界并转换它
        # 对于通常我们所做的格式（x，y，w，h）
        # OPEN与OpenCV
	x = rect.left()
	y = rect.top()
	w = rect.right() - x
	h = rect.bottom() - y
 
	# 返回 tuple  (x, y, w, h)
	return (x, y, w, h)
 
def shape_to_np(shape, dtype="int"):
	# 初始化（x，y）坐标的列表
	coords = np.zeros((shape.num_parts, 2), dtype=dtype)
 
        # 对所有面部标志进行循环并转换
        # 关于x（y，y）坐标的2元组
	for i in range(0, shape.num_parts):
		coords[i] = (shape.part(i).x, shape.part(i).y)
 
	# 返回（x，y）坐标的列表
	return coords
 
def visualize_facial_landmarks(image, shape, colors=None, alpha=0.75):
        # 创建输入图像的两个副本 (一个用于输入图像)
        # 叠加和 (一个用于最终输出图像)
	overlay = image.copy()
	output = image.copy()
 
        # 如果颜色列表为“否”，则用唯一的方法初始化它。
        # 每个脸部标志区域的颜色
	if colors is None:
		colors = [(19, 199, 109), (79, 76, 240), (230, 159, 23),
			(168, 100, 168), (158, 163, 32),
			(163, 38, 32), (180, 42, 220)]
 
	# 在面部标志区域上分别的连线
	for (i, name) in enumerate(FACIAL_LANDMARKS_IDXS.keys()):
		# grab the (x, y)-coordinates associated with the
		# face landmark
		(j, k) = FACIAL_LANDMARKS_IDXS[name]
		pts = shape[j:k]
 
		# 检查是否应该画下颌线
		if name == "jaw":
			# 因为 jawline 是一个非封闭的面部区域，
                        # 只画出（x，y）坐标之间的直线
			for l in range(1, len(pts)):
				ptA = tuple(pts[l - 1])
				ptB = tuple(pts[l])
				cv2.line(overlay, ptA, ptB, colors[i], 2)
 
		# 处理计算点的凸包
		# 并在覆盖图上绘制船体
		else:
			hull = cv2.convexHull(pts)
			cv2.drawContours(overlay, [hull], -1, colors[i], -1)
 
	# 应用透明覆盖
	cv2.addWeighted(overlay, alpha, output, 1 - alpha, 0, output)
 
	# 返回输出图像
	return output

 
#初始化dlib人脸检测（基于HOG），然后创建面部标志预测器
detector = dlib.get_frontal_face_detector()
 
predictor = dlib.shape_predictor('./data/shape_predictor_68_face_landmarks.dat')
 
cap = cv2.VideoCapture(0)
while(True):
    # 一帧一帧捕捉
    ret, image = cap.read()
    # 我们对帧的操作在这里
    image = imutils.resize(image, width=500)  # 调整图片宽度为500
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)#图片调整为灰色
    rects = detector(gray, 1)
    for (i, rect) in enumerate(rects):
        shape = predictor(gray, rect)
        shape = shape_to_np(shape)
        ii=visualize_facial_landmarks(image,shape)
        cv2.imshow('img',ii)
    if len(rects) == 0:
	    cv2.imshow('img',image)
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
'''
眨眼闭嘴检测

https://huaweicloud.csdn.net/63808c5edacf622b8df8a83a.html?spm=1001.2101.3001.6650.6&utm_medium=distribute.pc_relevant.none-task-blog-2~default~BlogCommendFromBaidu~activity-6-126468733-blog-80366524.235^v28^pc_relevant_t0_download&depth_1-utm_source=distribute.pc_relevant.none-task-blog-2~default~BlogCommendFromBaidu~activity-6-126468733-blog-80366524.235^v28^pc_relevant_t0_download&utm_relevant_index=7
'''
from imutils import face_utils
import numpy as np
import dlib
import cv2

# 眼长宽比例
def eye_aspect_ratio(eye):
    # (|e1-e5|+|e2-e4|) / (2|e0-e3|)
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# 嘴长宽比例
def mouth_aspect_ratio(mouth):
    A = np.linalg.norm(mouth[1] - mouth[7])  # 61, 67
    B = np.linalg.norm(mouth[3] - mouth[5])  # 63, 65
    C = np.linalg.norm(mouth[0] - mouth[4])  # 60, 64
    mar = (A + B) / (2.0 * C)
    return mar

#  进行活体检测（包含眨眼和张嘴）
def liveness_detection():
    vs = cv2.VideoCapture(0)  # 调用第一个摄像头的信息

    # 眼长宽比例值
    EAR_THRESH = 0.15
    EAR_CONSEC_FRAMES_MIN = 1
    EAR_CONSEC_FRAMES_MAX = 5  # 当EAR小于阈值时，接连多少帧一定发生眨眼动作

    # 嘴长宽比例值
    MAR_THRESH = 0.2

    # 初始化眨眼的连续帧数
    blink_counter = 0
    # 初始化眨眼次数总数
    blink_total = 0
    # 初始化张嘴次数
    mouth_total = 0
    # 初始化张嘴状态为闭嘴
    mouth_status_open = 0

    print("[INFO] loading facial landmark predictor...")
    # 人脸检测器
    detector = dlib.get_frontal_face_detector()
    # 特征点检测器
    predictor = dlib.shape_predictor("data/shape_predictor_68_face_landmarks.dat")
    # 获取左眼的特征点
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    # 获取右眼的特征点
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
    # 获取嘴巴特征点
    (mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["inner_mouth"]

    print("[INFO] starting video stream thread...")
    while True:

        flag, frame = vs.read()  # 返回一帧的数据
        if not flag:
            print("不支持摄像头", flag)
            break

        if frame is not None:
            # 图片转换成灰色（去除色彩干扰，让图片识别更准确）
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            rects = detector(gray, 0)  # 人脸检测
            # 只能处理一张人脸
            if len(rects) == 1:
                shape = predictor(gray, rects[0])  # 保存68个特征点坐标的<class 'dlib.dlib.full_object_detection'>对象
                shape = face_utils.shape_to_np(shape)  # 将shape转换为numpy数组，数组中每个元素为特征点坐标

                left_eye = shape[lStart:lEnd]  # 取出左眼对应的特征点
                right_eye = shape[rStart:rEnd]  # 取出右眼对应的特征点
                left_ear = eye_aspect_ratio(left_eye)  # 计算左眼EAR
                right_ear = eye_aspect_ratio(right_eye)  # 计算右眼EAR
                ear = (left_ear + right_ear) / 2.0   # 求左右眼EAR的均值

                inner_mouth = shape[mStart:mEnd]  # 取出嘴巴对应的特征点
                mar = mouth_aspect_ratio(inner_mouth)  # 求嘴巴mar的均值
                left_eye_hull = cv2.convexHull(left_eye)  # 寻找左眼轮廓
                right_eye_hull = cv2.convexHull(right_eye)  # 寻找右眼轮廓
                mouth_hull = cv2.convexHull(inner_mouth)  # 寻找内嘴巴轮廓
                cv2.drawContours(frame, [left_eye_hull], -1, (0, 255, 0), 1)   # 绘制左眼轮廓
                cv2.drawContours(frame, [right_eye_hull], -1, (0, 255, 0), 1)  # 绘制右眼轮廓
                cv2.drawContours(frame, [mouth_hull], -1, (0, 255, 0), 1)  # 绘制嘴巴轮廓

                # EAR低于阈值，有可能发生眨眼，眨眼连续帧数加一次
                if ear < EAR_THRESH:
                    blink_counter += 1

                # EAR高于阈值，判断前面连续闭眼帧数，如果在合理范围内，说明发生眨眼
                else:
                    # if the eyes were closed for a sufficient number of
                    # then increment the total number of blinks
                    if EAR_CONSEC_FRAMES_MIN <= blink_counter <= EAR_CONSEC_FRAMES_MAX:
                        blink_total += 1
                    blink_counter = 0
                # 通过张、闭来判断一次张嘴动作
                if mar > MAR_THRESH:
                     mouth_status_open = 1
                else:
                    if mouth_status_open:
                        mouth_total += 1
                    mouth_status_open = 0

                cv2.putText(frame, "Blinks: {}".format(blink_total), (0, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, "Mouth: {}".format(mouth_total),
                            (130, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, "MAR: {:.2f}".format(mar), (450, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            elif len(rects) == 0:
                cv2.putText(frame, "No face!", (0, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                cv2.putText(frame, "More than one face!", (0, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.namedWindow("Frame", cv2.WINDOW_NORMAL)
            cv2.imshow("Frame", frame)
            # 按下q键退出循环（鼠标要点击一下图片使图片获得焦点）
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    cv2.destroyAllWindows()
    vs.release()

#  调用摄像头进行张嘴眨眼活体检测
liveness_detection()